# Import Libraries

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import re
import cv2
import pickle
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mss import mss
from tensorflow.keras.models import load_model

# Setup

### Set path to tesseract if not exported to `$PATH`

In [2]:
path_to_tesseract = 'Your:\\Path\\To\\Tesseract\\exe'
assert os.path.exists(path_to_tesseract)
pytesseract.pytesseract.tesseract_cmd = path_to_tesseract

### Ensure proper file structure, all files exist

In [3]:
assert os.path.exists('./screenshots'), 'Screenshots folder does nto exist!'
assert os.path.exists('./models'), 'Models folder does not exist!'
assert os.path.exists('./models/LoLWatcher V2/saved_model.pb'), 'Model does not exist!'
assert os.path.exists('./models/scaler.pkl'), 'Scaler does not exist!'
assert os.path.exists('./models/pixel_coords.pkl'), 'Pixel coords does not exist!'
assert os.path.exists('./models/champ_dict.pkl'), 'Champ dict does not exist!'

### Set monitor to capture

In [4]:
monitor = 2

Run this cell below if you need to test which monitor number to use. If this cell displays the intended display, continue.

In [ ]:
#Monitor test, displays sample output to ensure correct monitor setting
mss().shot(mon=monitor, output='./screenshots/monitor_test.png')
plt.figure(figsize=(16,8))
img = plt.imread('./screenshots/monitor_test.png')
plt.imshow(img)
plt.axis('off')
os.remove('./screenshots/monitor_test.png')

# Setup Draft

### Manually enter champs:
Drafted champions must be manually input into the list below. Most champions are simply their name as they appear in game (first letter capitalized).
Ex. Annie, Olaf, Galio

Some notable exceptions:
1. Aurelion Sol > AurelionSol
1. Bel'Veth > Belveth
1. Cho'Gath > Chogath
1. Dr. Mundo > DrMundo
1. Jarvan IV > JarvanIV
1. Kai'Sa > Kaisa
1. K'Sante > KSante
1. Kha'Zix > Khazix
1. Kog'Maw > KogMaw
1. LeBlanc > Leblanc
1. Lee Sin > LeeSin
1. Master Yi > MasterYi
1. Miss Fortune > MissFortune
1. Nunu & Willump > Nunu
1. Rek'Sai > RekSai
1. Renata Glasc > Renata
1. Tahm Kench > TahmKench
1. Twisted Fate > TwistedFate
1. Vel'Koz > Velkoz
1. Wukong > MonkeyKing
1. Xin Zhao > XinZhao

In [6]:
champ_draft_list = [
    'Fiora', #Blue Top
    'Poppy', #Blue Jungle
    'Taliyah', #Blue Mid
    'Zeri', #Blue AD
    'Alistar', #Blue Support
    'Gnar', #Red Top
    'MonkeyKing', #Red Jungle
    'Zilean', #Red Mid
    'Sivir', #Red AD
    'Lulu' #Red support
]

#Should be in order of blue top -> jungle -> mid -> ad -> support -> red top -> support

### Variable initialization and helper functions
Do not edit

In [7]:
def reset_match_info():
    """Reset match_info dict start of game state, only champion ids will remain

    Returns:
        dict: match_info with cleared state
    """
    return {
    'timer': 0,
    'blue_team_barons': 0,
    'red_team_barons': 0,
    'blue_team_dragons': 0,
    'red_team_dragons': 0,
    'blue_team_kills': 0,
    'red_team_kills': 0,
    'blue_team_tower': 0,
    'red_team_tower': 0,
    'blue_team_first_tower': 0,
    'red_team_first_tower': 0,
    'player_0_champ': champ_id_list[0],
    'player_0_cs': 0,
    'player_0_k': 0,
    'player_0_d': 0,
    'player_0_a': 0,
    'player_1_champ': champ_id_list[1],
    'player_1_cs': 0,
    'player_1_k': 0,
    'player_1_d': 0,
    'player_1_a': 0,
    'player_2_champ': champ_id_list[2],
    'player_2_cs': 0,
    'player_2_k': 0,
    'player_2_d': 0,
    'player_2_a': 0,
    'player_3_champ': champ_id_list[3],
    'player_3_cs': 0,
    'player_3_k': 0,
    'player_3_d': 0,
    'player_3_a': 0,
    'player_4_champ': champ_id_list[4],
    'player_4_cs': 0,
    'player_4_k': 0,
    'player_4_d': 0,
    'player_4_a': 0,
    'player_5_champ': champ_id_list[5],
    'player_5_cs': 0,
    'player_5_k': 0,
    'player_5_d': 0,
    'player_5_a': 0,
    'player_6_champ': champ_id_list[6],
    'player_6_cs': 0,
    'player_6_k': 0,
    'player_6_d': 0,
    'player_6_a': 0,
    'player_7_champ': champ_id_list[7],
    'player_7_cs': 0,
    'player_7_k': 0,
    'player_7_d': 0,
    'player_7_a': 0,
    'player_8_champ': champ_id_list[8],
    'player_8_cs': 0,
    'player_8_k': 0,
    'player_8_d': 0,
    'player_8_a': 0,
    'player_9_champ': champ_id_list[9],
    'player_9_cs': 0,
    'player_9_k': 0,
    'player_9_d': 0,
    'player_9_a': 0,
    'blue_team_gold': 0,
    'red_team_gold': 0,
    }
    
def reset_df():
    """Reset df for game tracking, removes all game data while preserving columns

    Returns:
        DataFrame: Pandas DataFrame with appropriate columns and no data
    """
    return pd.DataFrame(columns=reset_match_info().keys())

#Set Tesseract OCR Config
config = '-l eng --psm 7 --oem 1'

#Unpickle dict for converting champ name to id
with open('./models/champ_dict.pkl', 'rb') as f:
    champ_dict = pickle.load(f)

#Unpickle dict for converting graphics package coordinates for specific information
with open('./models/pixel_coords.pkl', 'rb') as f:
    pixel_coords = pickle.load(f)

#Unpickle sci-kit StandardScaler for scaling data
with open('./models/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
        
#Load predictive TensorFlow model
model = load_model('./models/LoLWatcher V2')

#Convert champ names list to champ ids list
champ_id_list = [champ_dict[x] for x in champ_draft_list]

#Reset predictions list
predictions = []

#Reset tracking DataFrame df and match_info
df = reset_df()
match_info = reset_match_info()

# Output

Run this cell with the game running on the desired monitor.
Cell operation can be interrupted to stop the process.

In [11]:
try: #Prevent KeyboardInterrupt error log when stopping cell
    while (True):
        #Screenshot specified monitor
        mss().shot(mon=monitor, output='./screenshots/current.png')
        #Read image using openCV, resize to 1920x1080 resolution for pixel mapping
        img = cv2.imread('./screenshots/current.png')
        img = cv2.resize(img, (1920, 1080))
        #Iterate through known locations for text
        for key, value in pixel_coords.items():
            #Crop image to specific location
            cropped = img[value[0]:value[1], value[2]:value[3]]
            if 'kda' in key:
                #Parse text, apply regex filter for numeric values in k/d/a format
                text = re.findall(r'(?:\d{1,2}/){2}(?:\d{1,2})', pytesseract.pytesseract.image_to_string(cropped, config=config))
                if (len(text) > 0): #If specific text detected, set k,d,a columns
                    text = text[0].split('/')
                    match_info[key[:-4] + '_k'] = text[0]
                    match_info[key[:-4] + '_d'] = text[1]
                    match_info[key[:-4] + '_a'] = text[2]
            elif 'cs' in key or 'kills' in key:
                #Parse text, apply regex filter for numeric values
                text = re.findall(r'\d+', pytesseract.pytesseract.image_to_string(cropped, config=config))
                if (len(text) > 0): #If numerical, set cs columns
                    match_info[key] = text[0]
            elif 'gold' in key:
                #Parse text, apply regex filter for numeric values
                gold = re.findall(r'\d+', pytesseract.pytesseract.image_to_string(cropped, config=config))
                if (len(gold) > 1): #If numeric and thousands and hundreds digits are read correctly, set gold columns
                    match_info[key] = int(gold[0]) * 1000 + int(gold[1]) * 100
            elif 'team_tower' in key:
                #Parse text, apply regex filter for numerical values
                text = re.findall(r'\d+', pytesseract.pytesseract.image_to_string(cropped, config=config))
                if len(text) > 0: #If numeric, set number of towers taken
                    match_info[key] = text[0]
            elif 'timer' in key:
                #Parse text, preserve unformatted time in mm:ss format for output
                game_time = pytesseract.pytesseract.image_to_string(cropped, config=config).strip()
                #Parse text, apply regex filter for numeric values
                timer = re.findall(r'\d+', pytesseract.pytesseract.image_to_string(cropped, config=config))
                #Convert to int type
                timer = [int(x) for x in timer]
                if len(timer) > 1: #If minutes and seconds detected, convert to seconds
                    match_info[key] = timer[0] * 60 + timer[1]
            
        #Parse objective text in center of screen
        objective = pytesseract.pytesseract.image_to_string(img[200:270, 400:1520], config=config).lower()
        #If a team name is detected in objective text, find the appropriate counter and increment
        if 'blue' in objective:
            if 'baron' in objective:
                match_info['blue_team_barons'] += 1
            elif 'dragon' in objective:
                match_info['blue_team_dragons'] += 1
            elif 'first' in objective and 'turret' in objective:
                match_info['blue_team_first_tower'] += 1
        elif 'red' in objective:
            if 'baron' in objective:
                match_info['red_team_barons'] += 1
            elif 'dragon' in objective:
                match_info['red_team_dragons'] += 1
            elif 'first' in objective and 'turret' in objective:
                match_info['red_team_first_tower'] += 1
                
        #Append the current info from the screenshot to the tracking DataFrame df
        temp = pd.DataFrame.from_dict([match_info])
        df = pd.concat([df, temp], ignore_index=True)
        
        #Make a copy of the tracking DataFrame df and take the last row
        X = df.copy()
        X = X.tail(1)
        #Scale the copy using the pre-fitted StandardScaler
        X = scaler.transform(X)

        #Make a prediction using the model, convert to percentage value
        prediction = model.predict(X, verbose=0)[0][0] * 100
        #Append prediction to list of predictions
        predictions.append(prediction)
        #Print the prediction
        print(' ' * 75, end='\r')
        print(f'At {game_time} in game, Blue side has a {prediction:.2f}% chance of  winning.', end='\r')
except KeyboardInterrupt:
    #If run is stopped by interrupting kernel, print the final prediction
    print(' ' * 75, end='\r')
    print(f'Prediction Ended, final prediction was {prediction:.2f}%.')

Prediction Ended                                                           


(Model is red side favored, especially early)

### Plot prediction over time

Likely not fully accurate, high possibility of wrong/missing data.

In [ ]:
#Make a copy of the tracking DataFrame df
df_graph = df.copy()
#Insert the predictions
df_graph.insert(1, 'prediction', pd.Series(predictions))
#Take only the relevant X and y values
df_graph = df_graph[['timer', 'prediction']]

#Plot time vs prediction
plt.figure(figsize=(20,10))
plt.plot(df_graph['timer'], df_graph['prediction']);
plt.xlabel('Timer (s)');
plt.ylabel('Blue % Chance To Win');
plt.ylim(0, 100);